## Load data from author json file

In [ ]:
import logging
from tqdm.auto import tqdm
from pymilvus import connections, utility, Collection
from multiprocessing import Pool
from embedding_search.vector_store import (
    AUTHOR_DIR,
    create_authors_collection,
    create_articles_collection,
    make_author_data_package,
    make_articles_data_packages,
)

logging.basicConfig(level=logging.DEBUG)

### Connect to Milvus

In [ ]:
connections.connect(host="milvus-standalone", port="19530")

## Make Milvus collections

Init collection and load data from json file

In [ ]:
create_authors_collection()
create_articles_collection()

List collections

In [ ]:
utility.list_collections()

In [ ]:
author_collection = Collection("authors")
print(f"There are {author_collection.num_entities} authors in the DB.")

article_collection = Collection("articles")
print(f"There are {article_collection.num_entities} articles in the DB.")

## Ingest data

Authors

In [ ]:
author_ids = [path.stem for path in AUTHOR_DIR.glob("*.json")]
with Pool(8) as p:
    data_packages = p.map(make_author_data_package, author_ids)

In [ ]:
author_collection.insert(data_packages)
author_collection.flush()

Articles

In [ ]:
author_ids = [path.stem for path in AUTHOR_DIR.glob("*.json")]

article_collection = Collection("articles")
for author_id in tqdm(author_ids):
    data_packages = make_articles_data_packages(author_id)
    article_collection.insert(data_packages)

In [ ]:
article_collection.flush()

## Create Milvus index

In [ ]:
index_params = {
    "metric_type": "IP",  # inner product
    "index_type": "IVF_FLAT",
    "params": {"nlist": 1024},
}

In [ ]:
article_collection.create_index("embedding", index_params)

In [ ]:
utility.index_building_progress("articles")

## Load collection and test search

In [ ]:
article_collection.load()

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
search_vector = embeddings.embed_query("Dark Higgs's boson")

In [ ]:
search_params = {"metric_type": "IP", "params": {"nprobe": 16}}
articles = article_collection.search(
    data=[search_vector],
    anns_field="embedding",
    param=search_params,
    limit=2,
    output_fields=["title", "author_id"],
)

In [ ]:
articles[0][0]

Get only the title

In [ ]:
articles[0][0].entity.get("title")

## Drop collection

In [ ]:
utility.list_collections()

In [ ]:
# utility.drop_collection("articles")